# 2D flying-snake at Re=2000 and AoA=30deg (shifted markers)

In [1]:
import os
import sys
import collections

import ipywidgets

import snake
from snake import miscellaneous
from snake.petibm.simulation import PetIBMSimulation
from snake.cuibm.simulation import CuIBMSimulation

In [2]:
print('Python version: ' + sys.version)
print('snake version:' + snake.__version__)

Python version: 2.7.11 |Anaconda 2.5.0 (64-bit)| (default, Dec  6 2015, 18:08:32) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
snake version:0.1


---

### Description

Flying-snake at Re=2000 and angle-of-attack 30deg.

We compare the PetIBM results with those from Krishnan et al. (2014).

---

### General information

* software: `PetIBM`
* version: branch `master`, tag `snake-repro-1`
* dependencies: `PETSc-3.5.2`, `OpenMPI-1.8`, and `GCC-4.9.2`
* submission date: 2015/08/07 (mesnardo)
* running date: 2015/08/07
* machine: `Colonial One` (short queue, 32 processes, 2 nodes)

In [ ]:
simulation = PetIBMSimulation(description='PetIBM')

---

### Computational domain and mesh

We use the same input file than Krishnan et al. (2014).

In [ ]:
domain_info = os.path.join(simulation.directory, 'cartesianMesh.yaml')
%cat $domain_info

---

### Boundary conditions

We use the same input file than Krishnan et al. (2014).

In [ ]:
boundary_info = os.path.join(simulation.directory, 'flowDescription.yaml')
%cat $boundary_info

---

### Simulation parameters

- start / end / saving / time-increment: 0 / 80 / 1 / 0.0004
- method: Immersed-boundary projection method (Taira and Colonius, 2007)
- linear solvers:
    + velocity: CG (relative-tolerance: 1.0E-05) with diagonal preconditioner
    + Poisson: biCGStab (relative-tolerance: 1.0E-05) with smooth aggregation preconditioner

In [ ]:
simulation_info = os.path.join(simulation.directory, 'simulationParameters.yaml')
%cat $simulation_info

---

### Running the case

This simulation was run on our university HPC cluster, Colonial One.

To run the simulation (from the simulation directory):
    > mpirun -n <nprocs> <path to petibm2d executable> \
            -caseFolder <simulation direcotry> \
            -sys2_ksp_type bcgs \
            -sys2_pc_type gamg \
            -sys2_pc_gamg_type agg \
            -sys2_pc_gamg_agg_nsmooths 1 \
            -log_summary

---

### Post-processing

We computed 80 non-dimensional time-units in about 40 hours.

#### Forces

In [ ]:
simulation.read_forces()
simulation.get_mean_forces(limits=[32.0, 64.0])
simulation.get_strouhal(limits=[32.0, 64.0], order=200)

simulation.plot_forces(display_coefficients=True, 
                       coefficient=2.0,
                       display_extrema=True, order=200,
                       limits=(0.0, 80.0, 0.0, 3.0),
                       style='snakeReproducibility',
                       save_name='forceCoefficients')
dataframe = simulation.create_dataframe_forces(display_strouhal=True,
                                               display_coefficients=True,
                                               coefficient=2.0)

In [ ]:
krishnan = CuIBMSimulation(description='Krishnan et al. (2014)')
krishnan.read_forces(file_path=os.path.join(os.environ['SNAKE'],
                                            'resources',
                                            'flyingSnake2d_cuibm_anush',
                                            'flyingSnake2dRe2000AoA30',
                                            'forces'))
krishnan.get_mean_forces(limits=[32.0, 64.0])
krishnan.get_strouhal(limits=[32.0, 64.0], order=200)

simulation.plot_forces(display_coefficients=True, 
                       coefficient=2.0,
                       display_extrema=True, order=200,
                       limits=(0.0, 80.0, 0.0, 3.0),
                       other_simulations=krishnan, 
                       other_coefficients=2.0,
                       style='snakeReproducibility',
                       save_name='forceCoefficientsCompareKrishnanEtAl2014')
dataframe2 = krishnan.create_dataframe_forces(display_strouhal=True,
                                              display_coefficients=True,
                                              coefficient=2.0)

In [ ]:
dataframes = dataframe.append(dataframe2)
print(dataframes)

In [ ]:
images_directory = os.path.join(simulation.directory,
                                'images')
figures = collections.OrderedDict()
figures['present'] = os.path.join(images_directory,
                                  'forceCoefficients.png')
figures['present + Krishnan et al. (2014)'] = os.path.join(images_directory,
                                                           'forceCoefficientsCompareKrishnanEtAl2014.png')
ipywidgets.interact(miscellaneous.display_image, figure=figures);

#### Vorticity field

In [ ]:
simulation.read_grid()
time_steps = simulation.get_time_steps()
for time_step in time_steps:
    simulation.read_fields('vorticity', time_step)
    simulation.plot_contour('vorticity',
                            field_range=[-5.0, 5.0, 101],
                            filled_contour=True,
                            view=[-2.0, -5.0, 15.0, 5.0],
                            width=8.0,
                            time_increment=0.0004,
                            style='snakeReproducibility')

In [ ]:
miscellaneous.displayer(os.path.join(simulation.directory, 
                                     'images', 
                                     'vorticity_-2.00_-5.00_15.00_5.00'))